In [2]:
pwd

'/Users/xduplm/Library/CloudStorage/GoogleDrive-marceldpl10@gmail.com/My Drive/projects/2023_duplessis_storms_fluxes/duplessis-storms-warming/duplessis-storms-warming/code'

In [5]:
# saved SAF and PF locations from Sokolov and Rintoul (2009)

import scipy.io
import numpy as np

mat = scipy.io.loadmat("../data/fronts/ACCfronts.mat")

XPF  = np.array(list(mat.items())[3][1]).squeeze()
XSAF = np.array(list(mat.items())[4][1]).squeeze()
YPF  = np.array(list(mat.items())[5][1]).squeeze()
YSAF = np.array(list(mat.items())[6][1]).squeeze()

import pandas as pd

XPF  = pd.Series(XPF )
XSAF = pd.Series(XSAF)
YPF  = pd.Series(YPF )
YSAF = pd.Series(YSAF)

window_size = 10

XPF  = XPF .rolling(window=window_size, min_periods=1).mean()
XSAF = XSAF.rolling(window=window_size, min_periods=1).mean()
YPF  = YPF .rolling(window=window_size, min_periods=1).mean()
YSAF = YSAF.rolling(window=window_size, min_periods=1).mean()

# load the sea ice concentration data set

import xarray as xr

sic_noaa = xr.open_dataset('../data/seaice/icec.mnmean.nc')

# Step 1: Group by month and calculate the mean
sic_month = sic_noaa.groupby('time.month').mean(dim='time')

# Step 2: Select December values (month=12)
sic_dec = sic_month.sel(month=12).sel(lat=slice(0, -90))

lat_sie = np.ndarray(sic_dec.lon.size)

for i in range(sic_dec.lon.size):

    x = sic_dec.isel(lon=i)

    lat_sie[i] = x.icec[x.icec > 15].lat.max()

# smooth teh sea ice

idx1 = sic_dec.lon >= 180

idx2 = sic_dec.lon < 180

sie_lon = np.append(sic_dec.lon[idx1].values-360, sic_dec.lon[idx2].values)

sie_lat = np.append(lat_sie[idx1], lat_sie[idx2])

import numpy as np

def circular_running_mean(x, N):
    padded_x = np.concatenate((x[-N+1:], x, x[:N-1]))
    cumsum = np.cumsum(np.insert(padded_x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)


sie_lat = circular_running_mean(sie_lat, 3)[1:-1]

In [9]:
fr_df = pd.DataFrame(data={'pf_lon': XPF,
                           'pf_lat': YPF,
                           'saf_lon': XSAF,
                           'saf_lat': YSAF
                           })

si_df = pd.DataFrame(data={'si_lon': sie_lon,
                           'si_lat': sie_lat
                           })

In [10]:
fr_df.to_csv('../data/fronts/ACCfronts.csv')
si_df.to_csv('../data/seaice/sie.csv')